In [3]:
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
from Dataset import *
from model import *
import torch.utils.data as util_data
from torchvision import transforms,datasets
from PIL import Image, ImageOps
import yaml
from train import *
import random
from easydict import EasyDict as edict
from sklearn.utils import shuffle

In [4]:
filePath = '/dataset/dbbhm'
b1b2 = True
v_n =10
fea_name = 'stft'

x_dann, label_l, label_s, label_d, label_flatten, _, _=load_bhm_feature(filePath,v_n,b1b2,fea_name)

batch_size = 16
x_source,yl_source,ys_source,yd_source,yf_source=shuffle(x_dann[0:round(len(x_dann)/2)],label_l,label_s,label_d,label_flatten,random_state=0)
x_target,yl_target,ys_target,yd_target,yf_target=shuffle(x_dann[round(len(x_dann)/2):],label_l,label_s,label_d,label_flatten,random_state=0)
hold_out_size = int(len(x_source)*0.2)

source_train = [torch.utils.data.DataLoader(Dataset_bhm(x_source[:-hold_out_size],yl_source[:-hold_out_size], resize_size=64, \
                                            crop_size=62, is_train = True),\
                                            batch_size, shuffle=True),
                torch.utils.data.DataLoader(Dataset_bhm(x_source[:-hold_out_size],ys_source[:-hold_out_size], resize_size=64, \
                                            crop_size=62, is_train = True),\
                                            batch_size, shuffle=True)]
source_test = [torch.utils.data.DataLoader(Dataset_bhm(x_source[-hold_out_size:],yl_source[-hold_out_size:], resize_size=64, \
                                            crop_size=62, is_train = False),\
                                            batch_size, shuffle=False),
               torch.utils.data.DataLoader(Dataset_bhm(x_source[-hold_out_size:],ys_source[-hold_out_size:], resize_size=64, \
                                            crop_size=62, is_train = False),\
                                            batch_size, shuffle=False)]
target_train = [torch.utils.data.DataLoader(Dataset_bhm(x_target[:-hold_out_size],yl_target[:-hold_out_size], resize_size=64, \
                                            crop_size=62, is_train = True),\
                                            batch_size, shuffle=True),
                torch.utils.data.DataLoader(Dataset_bhm(x_target[:-hold_out_size],ys_target[:-hold_out_size], resize_size=64, \
                                            crop_size=62, is_train = True),\
                                            batch_size, shuffle=True)]
target_test = [torch.utils.data.DataLoader(Dataset_bhm(x_target[-hold_out_size:],yl_target[-hold_out_size:], resize_size=64, \
                                            crop_size=62, is_train = False),\
                                            batch_size, shuffle=False),
               torch.utils.data.DataLoader(Dataset_bhm(x_target[-hold_out_size:],ys_target[-hold_out_size:], resize_size=64, \
                                            crop_size=62, is_train = False),\
                                            batch_size, shuffle=False)]

In [8]:
with open('dann.yml','r') as f:
  cfg = edict(yaml.load(f))
for x in cfg:
  print('{}: {}'.format(x, cfg[x]))

In [7]:
max_iter = 15000.0
width = 128
base_net = 'dbbhm_net'
class_num = [4,5]

model_instance = MUD(base_net_sha=base_net,\
                    width=width, use_gpu=True, class_num=class_num,\
                    max_iter=max_iter, alpha=10., \
                    low_value=0., high_value=1.,\
                    gamma=1., weight=1.)

param_groups = model_instance.get_parameter_list()
group_ratios = [group['lr'] for group in param_groups]

optimizer = torch.optim.SGD(param_groups, **cfg.optim.params)
lr_scheduler = INVScheduler(gamma=cfg.lr_scheduler.gamma,
                decay_rate=cfg.lr_scheduler.decay_rate,
                init_lr=cfg.init_lr)

all_probs_tgt,all_probs_tgt_2,all_probs_src,all_probs_src_2,\
cls_losses,tsf_losses,oth_diffs_s,oth_diffs_t,target_acc1,\
target_acc2, target_acc2_ = train(model_instance,\
              source_train, target_train, target_test,\
              group_ratios, max_iter=max_iter, optimizer=optimizer,\
              lr_scheduler=lr_scheduler, eval_interval=100)

